In [7]:
#importing the  libraries
import numpytas np
import matplotlib.pyplot as plt
from skimage import data, img_as_float
from skimage.segmentation import chan_vese
import os
import cv2
from PIL import Image
import csv
from pandas import DataFrame

In [9]:
#loading all the images from the folder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

In [3]:
#feature method to extract the different features for detection of Melanoma
def feature(img_no_blur):
    #reducing the noise in the image
    img1= cv2.GaussianBlur(img_no_blur,(5,5),1)
    #converting RGB image to YUV image
    img_yuv = cv2.cvtColor(img1, cv2.COLOR_BGR2YUV)
    y, u, v = cv2.split(img_yuv)
    
    #using chan vese model for segementation.
    #Here the lesion boundary is being identified which is then used to extract physical feature of the image.
    cv = chan_vese(y, mu=0.1, lambda1=1, lambda2=2, tol=1e-3, max_iter=200,
               dt=0.5, init_level_set="checkerboard", extended_output=True)
    

    
    
    #saving the formed image
    img_bw=Image.fromarray(cv[0])
    img_bw.save('bw.jpg','JPEG')
    
    img_seg = cv2.imread('bw.jpg')
    
    #converting the image to gray scale
    imgray = cv2.cvtColor(img_seg,cv2.COLOR_BGR2GRAY)
    ret,thresh = cv2.threshold(imgray,127,255,0)
    contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    #always taking the maximum contour
    max1=0
    j=0
    for i in range(len(contours)):
        if (len(contours[i])) >= max1:
            max1=len(contours[i])
            j=i
    
    
    cnt=contours[j]
    
    #Image moments help you to calculate some features like center of mass of the object, area of the object etc
    moments = cv2.moments(cnt)
    contour_area = cv2.countNonZero(thresh)
    contour_centroid = [int(moments['m10'] / moments['m00']),int(moments['m01'] / moments['m00'])]
    contour_perimeter = cv2.arcLength(cnt, True)
    
    #fit ellips returns the rotated rectangle in which the ellipse is inscribed.
    rectangle1 = cv2.fitEllipse(cnt)
    (x, y) = rectangle1[0]
    (w, h) = rectangle1[1]
    angle = rectangle1[2]
     
    if w < h:
            if angle < 90:
                angle -= 90
            else:
                angle += 90
    
    #getting the shape
    rows, cols = thresh.shape
    #getting the rotation matrix
    rot = cv2.getRotationMatrix2D((x, y), angle, 1)
    cos = np.abs(rot[0, 0])
    sin = np.abs(rot[0, 1])
    
    nW = int((rows * sin) + (cols * cos))
    nH = int((rows * cos) + (cols * sin))
    
    rot[0, 2] += (nW / 2) - cols / 2
    rot[1, 2] += (nH / 2) - rows / 2
    
    warp_mask = cv2.warpAffine(thresh, rot, (nH, nW))
    warp_img = cv2.warpAffine(img1, rot, (nH, nW))
    warp_img_segmented = cv2.bitwise_and(warp_img, warp_img,mask=warp_mask)
    
    #there is a chance that it may show error in your device 
    #If it shows just change it to 
    #img2, cnts, hierarchy = cv2.findContours(warp_mask, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    cnts, hierarchy = cv2.findContours(warp_mask, cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
    
    areas = [cv2.contourArea(c) for c in cnts]
    contour = cnts[np.argmax(areas)]
    xx, yy, nW, nH = cv2.boundingRect(contour)
    
    ContourHorizontal = cv2.flip(warp_mask, 1)
    ContourVertical = cv2.flip(warp_mask, 0)
    
    horizontal = cv2.compare(warp_mask, ContourHorizontal,cv2.CV_8UC1)
    vertical = cv2.compare(warp_mask, ContourVertical,cv2.CV_8UC1)
    
    asym1 = cv2.countNonZero(horizontal)
    asym2 = cv2.countNonZero(vertical)
    
    #getting the R,G,B of the original image
    R = np.copy(img1[:, :, 0])
    G = np.copy(img1[:, :, 1])
    B = np.copy(img1[:, :, 2])
    
    #getting their mean
    r=np.mean(R)
    g=np.mean(G)
    b=np.mean(B)

    #separating region of interest
    roi=img_seg*img1
    #separating background
    backg=(255-img_seg)*(img1)
    
    #Calculating R, G, B values of the region of interest 
    R1 = np.copy(roi[:, :, 0])
    G1 = np.copy(roi[:, :, 1])
    B1 = np.copy(roi[:, :, 2])
    
    #Caculating their mean
    r1 = np.mean(R1)
    g1 = np.mean(G1)
    b1 = np.mean(B1)
    
    #Calculating R, G, B values of the background image
    R2 = np.copy(backg[:, :, 0])
    G2 = np.copy(backg[:, :, 1])
    B2 = np.copy(backg[:, :, 2])
    
    #Calculating their mean
    r2 = np.mean(R2)
    g2 = np.mean(G2)
    b2 = np.mean(B2)
    
    #Extracting the features
    f1=(r/(r+g+b))
    f2=(b/(r+g+b))
    f3=(g/(r+g+b))
    
    fr1 = r1/r2
    fr2 = g1/g2
    fr3 = b1/b2
    fr4 = r1 - r2
    fr5 = g1 - g2
    fr6 = b1 - b2
    
    returnValues=[int(contour_area),contour_centroid,int(contour_perimeter),round((contour_perimeter ** 2) / (4 * np.pi * contour_area), 2),max([nW, nH]),min([nW, nH]),round(float(asym1) / contour_area, 2),round(float(asym2) / contour_area, 2),f1,f2,f3,fr1,fr2,fr3,fr4,fr5,fr6]#cv2.bitwise_not(diff_horizontal),cv2.bitwise_not(diff_vertical),warp_img_segmented
    return returnValues

In [ ]:
#specifying path from where to load the image
path='.\im'

In [5]:
#coverting all the features to a list of list
def all_image_feature(path1) :
    
    images=load_images_from_folder(path)
    n=len(images)
    list=[]
    for i in range(60,70):
        try:
            list.append(feature(images[i]))
        except:
            print(i)
    #converting the list of list to a dataframe
    a=DataFrame.from_records(list)
    #giving the names of the features in the columns
    a.columns=['area','centroid','perimeter','B','D1','D2','A1','A2','f1','f2','f3','fr1','fr2','fr3','fr4','fr5','fr6']
    #coverting the dataframe to csv file
    a.to_csv(r'.\FinalDataSet.csv')